About the dataset


1.   news_url

2.   title
3.   source


2.   tweet_num








1:Fake news

0:real news

In [2]:
#importing the dependencies
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords#those words which doesnot add values to the context of the news
from nltk.stem.porter import PorterStemmer#used yo stem our words
from sklearn.feature_extraction.text import TfidfVectorizer #text into feature vectors/numbers
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

Data pre-processing

In [5]:
#loading the dataset to a pandas dataframe
news_dataset=pd.read_csv('/content/FakeNewsNet.csv.zip')

In [6]:
news_dataset.shape

(23196, 5)

In [7]:
#print the first five rows of the dataframe
news_dataset.head()

,title,news_url,source_domain,tweet_num,real
0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42,1
1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0,1
2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63,1
3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,20,1
4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,38,1


In [8]:
#checking the number of missing values in the dataset
news_dataset.isnull().sum()

,0
title,0
news_url,330
source_domain,330
tweet_num,0
real,0


In [9]:
#replacing the null values with empty string
news_dataset=news_dataset.fillna('')

In [12]:
#merging the source_domain and title and storing it in the new column called content
news_dataset['content']=news_dataset['source_domain']+' '+news_dataset['title']

In [13]:
print(news_dataset['content'])

0        toofab.com Kandi Burruss Explodes Over Rape Ac...
1        www.today.com People's Choice Awards 2018: The...
2        www.etonline.com Sophia Bush Sends Sweet Birth...
3        www.dailymail.co.uk Colombian singer Maluma sp...
4        www.zerchoo.com Gossip Girl 10 Years Later: Ho...
                               ...                        
23191    www.express.co.uk Pippa Middleton wedding: In ...
23192    hollywoodlife.com Zayn Malik & Gigi Hadid’s Sh...
23193    www.justjared.com Jessica Chastain Recalls the...
23194    www.intouchweekly.com Tristan Thompson Feels "...
23195    www.billboard.com Kelly Clarkson Performs a Me...
Name: content, Length: 23196, dtype: object


In [14]:
#separating the data & target
X=news_dataset.drop(columns='real',axis=1)
Y=news_dataset['real']

In [15]:
print(X)
print(Y)

                                                   title  \
0      Kandi Burruss Explodes Over Rape Accusation on...   
1      People's Choice Awards 2018: The best red carp...   
2      Sophia Bush Sends Sweet Birthday Message to 'O...   
3      Colombian singer Maluma sparks rumours of inap...   
4      Gossip Girl 10 Years Later: How Upper East Sid...   
...                                                  ...   
23191  Pippa Middleton wedding: In case you missed it...   
23192  Zayn Malik & Gigi Hadid’s Shocking Split: Why ...   
23193  Jessica Chastain Recalls the Moment Her Mother...   
23194  Tristan Thompson Feels "Dumped" After Khloé Ka...   
23195  Kelly Clarkson Performs a Medley of Kendrick L...   

                                                news_url  \
0      http://toofab.com/2017/05/08/real-housewives-a...   
1      https://www.today.com/style/see-people-s-choic...   
2      https://www.etonline.com/news/220806_sophia_bu...   
3      https://www.dailymail.co.uk/news

Stemming is:

Stemming is the process of reducing a word to its root words

eg:  actor, actress, acting -->act is the root word

In [16]:
port_stem=PorterStemmer()

In [17]:
#creating a function called stemming using the def keyword
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]',' ',content)#removes everything other than the alphabets and replaced by space
  stemmed_content=stemmed_content.lower()#all the alphabets to lower case
  stemmed_content=stemmed_content.split()#spitting the words and converted to a list
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]#removing stop words
  stemmed_content=' '.join(stemmed_content)
  return stemmed_content

In [18]:
news_dataset['content']=news_dataset['content'].apply(stemming)

In [19]:
print(news_dataset['content'])

0        toofab com kandi burruss explod rape accus rea...
1        www today com peopl choic award best red carpe...
2        www etonlin com sophia bush send sweet birthda...
3        www dailymail co uk colombian singer maluma sp...
4        www zerchoo com gossip girl year later upper e...
                               ...                        
23191    www express co uk pippa middleton wed case mis...
23192    hollywoodlif com zayn malik gigi hadid shock s...
23193    www justjar com jessica chastain recal moment ...
23194    www intouchweekli com tristan thompson feel du...
23195    www billboard com kelli clarkson perform medle...
Name: content, Length: 23196, dtype: object


In [20]:
#separating the data and the real
X=news_dataset['content'].values
Y=news_dataset['real'].values

In [21]:
print(X)

['toofab com kandi burruss explod rape accus real housew atlanta reunion video'
 'www today com peopl choic award best red carpet look'
 'www etonlin com sophia bush send sweet birthday messag one tree hill co star hilari burton breyton eva'
 ...
 'www justjar com jessica chastain recal moment mother boyfriend slap kick genit'
 'www intouchweekli com tristan thompson feel dump khlo kardashian refus let move la home exclus'
 'www billboard com kelli clarkson perform medley kendrick lamar humbl hit billboard music award']


In [22]:
print(Y)

[1 1 1 ... 1 0 1]


In [23]:
Y.shape

(23196,)

In [24]:
#converting the textual data to numerical data for the system to understand
vectorizer=TfidfVectorizer()#tf-term frquency(finds the number of repeating significant words) and idf inverse data frequency(non significant words)
vectorizer.fit(X)
X=vectorizer.transform(X)

In [25]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 246417 stored elements and shape (23196, 14435)>
  Coords	Values
  (0, 59)	0.2597097788538541
  (0, 651)	0.30737565657585053
  (0, 1707)	0.38749399933436923
  (0, 2460)	0.0508218665886508
  (0, 4193)	0.35506164978733906
  (0, 5891)	0.26124620267491605
  (0, 6647)	0.38749399933436923
  (0, 10223)	0.302158398468634
  (0, 10276)	0.23128301802698967
  (0, 10569)	0.2543303205934995
  (0, 12980)	0.2946734912504806
  (0, 13681)	0.21108934405813046
  (1, 729)	0.3083771474680274
  (1, 1101)	0.33844294892026705
  (1, 1916)	0.3875096512688879
  (1, 2219)	0.42251489448694257
  (1, 2460)	0.08159478582438673
  (1, 7432)	0.33284454859410584
  (1, 9476)	0.23697414575567807
  (1, 10347)	0.37702288594702094
  (1, 12946)	0.36817044163275126
  (1, 14274)	0.10189043319535993
  (2, 1201)	0.19486226938688445
  (2, 1530)	0.3779018214766305
  (2, 1711)	0.3321987353251551
  :	:
  (23194, 4387)	0.2692478125672807
  (23194, 5800)	0.2632849856372537
  (

In [26]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)#stratify equalises the proportion of real and fake news

Traning the model : Logistic Regression Model

In [27]:
# Traning the model : Logistic Regression Model
model = LogisticRegression()
model.fit(X_train,Y_train)

LogisticRegression()

Evaluation

Accuracy score

In [28]:
#accuracy score on the training data
X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(X_train_prediction,Y_train)

In [29]:
print('Accuracy score of the training data : ',training_data_accuracy)

Accuracy score of the training data :  0.8827333477042466


In [30]:
#accuracy score on the test data
X_test_prediction=model.predict(X_test)
test_data_accuracy=accuracy_score(X_test_prediction,Y_test)

In [31]:
print('Accuracy score of the test data : ',test_data_accuracy)

Accuracy score of the test data :  0.8525862068965517


Making a predictive System

In [36]:
X_new=X_test[0]
prediction=model.predict(X_new)
print(prediction)
if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[1]
The news is Fake


In [37]:
print(Y_test[0])

1
